# Cleaning data

## Setup

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime
from geopandas import GeoDataFrame
from shapely.geometry import Point
from dateutil.relativedelta import relativedelta
from datetime import datetime

## Import dataset and delete unneccessary variables

In [3]:
nc_file = 'data/merged.nc'

ds  = xr.open_dataset(nc_file)
albedo = ds['cdr_surface_albedo']
temperature = ds['cdr_surface_temperature']

In [4]:
albedo

<xarray.DataArray 'cdr_surface_albedo' (time: 21668, columns: 321, rows: 321)>
[2232692388 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1990-01-02T02:00:00 ... 2020-01-01
    longitude  (columns, rows) float32 ...
    latitude   (columns, rows) float32 ...
Dimensions without coordinates: columns, rows
Attributes:
    standard_name:          surface_albedo
    long_name:              NOAA CDR of surface broadband albedo
    units:                  1
    coverage_content_type:  physicalMeasurement

In [5]:
temperature

<xarray.DataArray 'cdr_surface_temperature' (time: 21668, columns: 321,
                                             rows: 321)>
[2232692388 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1990-01-02T02:00:00 ... 2020-01-01
    longitude  (columns, rows) float32 ...
    latitude   (columns, rows) float32 ...
Dimensions without coordinates: columns, rows
Attributes:
    standard_name:          surface_temperature
    long_name:              NOAA CDR of surface skin temperature
    units:                  K
    coverage_content_type:  physicalMeasurement

In [11]:
del nc_file
del ds

## Clean and filter **surface albedo** data, and convert into pandas GeoDataFrame

In [6]:
df_albedo = albedo.to_dataframe()

In [7]:
df_albedo_loc = df_albedo[
    (df_albedo['longitude'] >= 51.670) & 
    (df_albedo['longitude'] <= 57.725) & 
    (df_albedo['latitude'] >= -82.586) & 
    (df_albedo['latitude'] <= -81.762) & 
    (df_albedo['cdr_surface_albedo'] != 'NaN')]

In [8]:
geometry = [Point(xy) for xy in zip(df_albedo_loc['longitude'], df_albedo_loc['latitude'])]
gdf_albedo_loc = GeoDataFrame(df_albedo_loc, geometry=geometry)
gdf_albedo_loc = gdf_albedo_loc.drop(columns=['longitude', 'latitude'])

In [9]:
gdf_albedo_loc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 303352 entries, (Timestamp('1990-01-02 02:00:00'), 138, 189) to (Timestamp('2020-01-01 00:00:00'), 142, 189)
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   cdr_surface_albedo  112830 non-null  float32 
 1   geometry            303352 non-null  geometry
dtypes: float32(1), geometry(1)
memory usage: 5.9 MB


In [10]:
del albedo
del df_albedo
del df_albedo_loc
del geometry

## Clean and filter **surface temperature** data, and convert into pandas GeoDataFrame

In [12]:
df_temperature = temperature.to_dataframe()

In [14]:
df_temperature_loc = df_temperature[
    (df_temperature['longitude'] >= 51.670) & 
    (df_temperature['longitude'] <= 57.725) & 
    (df_temperature['latitude'] >= -82.586) & 
    (df_temperature['latitude'] <= -81.762) & 
    (df_temperature['cdr_surface_temperature'] != 'NaN')]

In [15]:
geometry = [Point(xy) for xy in zip(df_temperature_loc['longitude'], df_temperature_loc['latitude'])]
gdf_temperature_loc = GeoDataFrame(df_temperature_loc, geometry=geometry)
gdf_temperature_loc = gdf_temperature_loc.drop(columns=['longitude', 'latitude'])

In [16]:
gdf_temperature_loc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 303352 entries, (Timestamp('1990-01-02 02:00:00'), 138, 189) to (Timestamp('2020-01-01 00:00:00'), 142, 189)
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   cdr_surface_temperature  292610 non-null  float32 
 1   geometry                 303352 non-null  geometry
dtypes: float32(1), geometry(1)
memory usage: 5.9 MB


In [17]:
del temperature
del df_temperature
del df_temperature_loc
del geometry

## Merge temperature and albedo monthly mean into a dict

In [37]:
start_date = datetime(1990, 1, 1)
end_date = datetime(2019, 12, 1)

In [38]:
data = {}

while start_date <= end_date:
    date_str = str(start_date.strftime("%Y-%m"))
    
    try:
        albedo_monthly_mean = str(gdf_albedo_loc.loc[date_str, 'cdr_surface_albedo'].mean())
        temperature_monthly_mean = str(gdf_temperature_loc.loc[date_str, 'cdr_surface_temperature'].mean())
        if albedo_monthly_mean == 'nan' or temperature_monthly_mean == 'nan':
            pass
        else:
            data[date_str] = {}
            data[date_str]['albedo'] = albedo_monthly_mean
            data[date_str]['temperature'] = temperature_monthly_mean
    except:
        pass
    
    start_date = start_date + relativedelta(months=1)

In [39]:
data

{'1990-01': {'albedo': '0.9344236', 'temperature': '255.69727'},
 '1990-02': {'albedo': '0.8875952', 'temperature': '247.42142'},
 '1990-03': {'albedo': '0.92373705', 'temperature': '234.41136'},
 '1990-09': {'albedo': '0.8793889', 'temperature': '221.92047'},
 '1990-10': {'albedo': '0.94586504', 'temperature': '237.13995'},
 '1990-11': {'albedo': '0.88554853', 'temperature': '249.94563'},
 '1990-12': {'albedo': '0.9345198', 'temperature': '250.31046'},
 '1991-01': {'albedo': '0.9550576', 'temperature': '249.72719'},
 '1991-02': {'albedo': '0.8051386', 'temperature': '251.27615'},
 '1991-03': {'albedo': '0.8913885', 'temperature': '233.30334'},
 '1991-09': {'albedo': '0.8086207', 'temperature': '221.1873'},
 '1991-10': {'albedo': '0.88577974', 'temperature': '232.17581'},
 '1991-11': {'albedo': '0.90987027', 'temperature': '245.1425'},
 '1991-12': {'albedo': '0.95202184', 'temperature': '245.90887'},
 '1992-01': {'albedo': '0.94819206', 'temperature': '244.80562'},
 '1992-02': {'albedo

## Save dict of final data as json

In [41]:
import json
json = json.dumps(data)
f = open("final_mm_data.json","w")
f.write(json)
f.close()

## Convert Kelvin to Celsius and save as CSV

In [84]:
import csv

header = ['date', 'albedo', 'temperature']

with open('final_mm_data.csv', 'w', newline='') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(header)
    for entry in data:
        item = data[entry]
        albedo = float(item['albedo'])
        temperature = float(item['temperature']) - 273.15 # Convert Kelvin to Celsius
        line = [entry, albedo, temperature]
        csvwriter.writerow(line)

## Generate CSV with yearly mean values

In [88]:
start_date = datetime(1990, 1, 1)
end_date = datetime(2019, 12, 1)

In [89]:
data = {}

while start_date <= end_date:
    date_str = str(start_date.strftime("%Y"))
    
    try:
        albedo_yearly_mean = str(gdf_albedo_loc.loc[date_str, 'cdr_surface_albedo'].mean())
        temperature_yearly_mean = str(gdf_temperature_loc.loc[date_str, 'cdr_surface_temperature'].mean())
        if albedo_yearly_mean == 'nan' or temperature_yearly_mean == 'nan':
            pass
        else:
            data[date_str] = {}
            data[date_str]['albedo'] = albedo_yearly_mean
            data[date_str]['temperature'] = temperature_yearly_mean
    except:
        pass
    
    start_date = start_date + relativedelta(months=1)

In [90]:
data

{'1990': {'albedo': '0.918978', 'temperature': '228.22087'},
 '1991': {'albedo': '0.9083623', 'temperature': '229.53526'},
 '1992': {'albedo': '0.90335006', 'temperature': '227.48744'},
 '1993': {'albedo': '0.82843447', 'temperature': '228.51353'},
 '1994': {'albedo': '0.7592709', 'temperature': '226.95282'},
 '1995': {'albedo': '0.9270495', 'temperature': '228.03595'},
 '1996': {'albedo': '0.93050104', 'temperature': '230.7377'},
 '1997': {'albedo': '0.9196208', 'temperature': '228.85664'},
 '1998': {'albedo': '0.8686161', 'temperature': '227.9806'},
 '1999': {'albedo': '0.86842096', 'temperature': '227.62265'},
 '2000': {'albedo': '0.8166456', 'temperature': '226.64476'},
 '2001': {'albedo': '0.9655592', 'temperature': '225.86191'},
 '2002': {'albedo': '0.94338125', 'temperature': '226.80609'},
 '2003': {'albedo': '0.9218586', 'temperature': '229.20352'},
 '2004': {'albedo': '0.88424885', 'temperature': '231.34618'},
 '2005': {'albedo': '0.8916691', 'temperature': '228.95064'},
 '200

In [92]:
header = ['year', 'albedo', 'temperature']

with open('final_ym_data.csv', 'w', newline='') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(header)
    for entry in data:
        item = data[entry]
        albedo = float(item['albedo'])
        temperature = float(item['temperature']) - 273.15 # Convert Kelvin to Celsius
        line = [entry, albedo, temperature]
        csvwriter.writerow(line)